# 예제 7-4 KNN 분류 알고리즘 

### 기본 라이브러리 불러오기

In [31]:
import pandas as pd
import seaborn as sns

## Step 1 데이터 준비 

### load_dataset 함수를 사용하여 데이터프레임 변환

In [32]:
df = sns.load_dataset('titanic')

### 데이터 살펴보기

In [33]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Step 2 데이터 탐색/전처리 

### 데이터 자료형 확인 

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 63.0+ KB


deck열에는 유효값이 891개중 688개의 누락데이터가 있다. 따라서 deck열을 제거하고, 

승선도시를 나타내는 embark_town 열은 embarked 열과 동일한 의미를 갖기 때문에 중복을 없애기 위해 열 자체를 제거한다.

### deck 열과 embark_town 열 삭제

In [35]:
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

### age열에 나이 데이터가 없는 모든 행 삭제 

In [36]:
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
len(rdf)

714

### embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기

In [37]:
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [38]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
rdf['embarked'].fillna(most_freq, inplace=True)

## Step 3 분석에 사용할 속성 선택 

### 분석에 활용할 열(속성) 선택 

In [40]:
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


KNN 모형에 적용하기 위해 sex열과 embarked열의 범주형 데이터를 숫자형으로 변환한다. 이 과정을 더미 변수를 만든다고 하고 원핫인코딩이라고도 부른다.

### 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환 

In [41]:
onehot_sex = pd.get_dummies(ndf['sex'])

In [42]:
ndf = pd.concat([ndf, onehot_sex], axis=1)

concat() 함수로 생성된 더미 변수 열을 기존 데이터프레임에 연결한다.

In [43]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')

prefix='town' 옵션을 사용하여 열 이름에 접두어 'town'을 붙인다.

In [44]:
ndf = pd.concat([ndf, onehot_embarked], axis=1)

In [45]:
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

기존 sex, embarked 열 삭제

In [46]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


## Step 4 데이터셋 구분 - 훈련용/검증용 

### 속성(변수) 선택 

In [47]:
X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male',
        'town_C', 'town_Q', 'town_S']]
Y = ndf['survived']

### 설명 변수 데이터를 정규화(normalization) 

In [49]:
from sklearn import preprocessing

In [50]:
X = preprocessing.StandardScaler().fit(X).transform(X)

### train data와 test data로 구분 

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,
                                                   random_state=10)

In [54]:
print("train data 개수: ", X_train.shape)
print("test data 개수: ", X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


## Step 5 KNN 분류 모형 - sklearn 사용 

### sklearn 라이브러리에서 KNN 분류 모형 가져오기

In [55]:
from sklearn.neighbors import KNeighborsClassifier

### 모형 객체 생성(k=5로 설정)

In [56]:
knn = KNeighborsClassifier(n_neighbors=5)

### train data를 가지고 모형 학습

In [57]:
knn.fit(X_train, Y_train)

KNeighborsClassifier()

### test data를 가지고 y_hat을 예측(분류)

In [58]:
y_hat = knn.predict(X_test)

In [62]:
print(y_hat)
print(Y_test.values)

[0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 1 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0
 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1
 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1
 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1]
[0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1
 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1
 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0
 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1
 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0]


### 모형 성능 평가 - Confusion Matrix 계산

In [63]:
from sklearn import metrics

In [66]:
knn_matrix = metrics.confusion_matrix(Y_test, y_hat)
knn_matrix

array([[109,  16],
       [ 25,  65]], dtype=int64)

215명의 승객 중에서 미생존자를 정확히 예측한 TP는 109명, 미생존자를 생존자로 분류한 FP는 16명, 생존자를 미생존자로 분류한 FN은 25명, 생존자를 정확히 예측한 TN은 65명으로 확인된다.

### 모형 성능 평가 - 평가 지표 계산

In [68]:
knn_report = metrics.classification_report(Y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.81      0.87      0.84       125
           1       0.80      0.72      0.76        90

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215



f1-score 지표를 보면 미생존자(0) 예측의 정확도가 0.84, 생존자(1) 예측의 정확도는 0.76으로 예측 능력에 차이가 있다.